In [1]:
#coding:utf-8
import torch as t
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
import ipdb

In [2]:
import sys
sys.path.append('../../ATAE-LSTM')
import Ipynb_importer
from config import opt
from data.Embedding import Emb
from models.BasicModule import BasicModule
from data.AspClas import AspClas

importing Jupyter notebook from ../../ATAE-LSTM\data\Embedding.ipynb
importing Jupyter notebook from ../../ATAE-LSTM\models\BasicModule.ipynb
importing Jupyter notebook from ../../ATAE-LSTM\data\AspClas.ipynb


In [3]:
class ATAE_LSTM(BasicModule):
    def __init__(self, emb):
        super(ATAE_LSTM, self).__init__()
        
        self.embedding = emb._make_layer_()
        
        self.lstm = nn.LSTM(opt.hidden_size*2, opt.hidden_size, batch_first=True)
        for k in self.lstm.state_dict().keys():
            self.lstm.state_dict()[k].uniform_(-opt.epsilon, opt.epsilon)
        
        """
        self.hidden=(
            # 三个参数分别为 num_layers, batch_size, hidden_size
            t.nn.Parameter(
                t.Tensor(
                    np.random.uniform(-opt.epsilon, opt.epsilon, opt.hidden_size)
                ).view(1,1,opt.hidden_size)
            ),
            t.nn.Parameter(
                t.Tensor(
                    np.random.uniform(-opt.epsilon, opt.epsilon, opt.hidden_size)
                ).view(1,1,opt.hidden_size)
            )
        )"""
        self.h0 = t.nn.Parameter(t.Tensor(
            np.random.uniform(-opt.epsilon, opt.epsilon, [1, opt.batch_size, opt.hidden_size])
        ))
        self.c0 = t.nn.Parameter(t.Tensor(
            np.random.uniform(-opt.epsilon, opt.epsilon, [1, opt.batch_size, opt.hidden_size])
        ))
        
        self.Wh = t.nn.Parameter(
            t.Tensor(
                np.random.uniform(-opt.epsilon, opt.epsilon, [opt.hidden_size, opt.hidden_size])
            )
        )
        self.Wv = t.nn.Parameter(
            t.Tensor(
                np.random.uniform(-opt.epsilon, opt.epsilon, [opt.hidden_size, opt.hidden_size])
            )
        )
        
        self.tanh = t.nn.Tanh()
        
        self.omega = t.nn.Parameter(
            t.Tensor(
                np.random.uniform(-opt.epsilon, opt.epsilon, opt.hidden_size*2)
            ).view(opt.hidden_size*2, 1)
        )
        
        self.Wp = t.nn.Parameter(
            t.Tensor(
                np.random.uniform(-opt.epsilon, opt.epsilon, [opt.hidden_size, opt.hidden_size])
            )
        )
        self.Wx = t.nn.Parameter(
            t.Tensor(
                np.random.uniform(-opt.epsilon, opt.epsilon, [opt.hidden_size, opt.hidden_size])
            )
        )
        self.relu = t.nn.LeakyReLU()
        
        self.lin = nn.Linear(opt.hidden_size, opt.classes)
        for k in self.lin.state_dict().keys():
            self.lin.state_dict()[k].uniform_(-opt.epsilon, opt.epsilon)
        
        return
    
    def forward(self, sentence, term):
        
        # sentence: [batch_size, max_seq_len]
        # term: [batch_size, max_term_len]
        
        sentence_embeddings = self.embedding(sentence) # [batch_size, max_seq_len, hidden_size]
        term_embeddings = []
        for batch in range(term.size(0)):
            valid_length = (term[batch, :] != 0).sum()
            term_embeddings.append(self.embedding(term[batch, :valid_length]).mean(dim=0).unsqueeze(dim=0))
        aspect_embeddings = t.cat(term_embeddings, dim=0) # [batch_size, hidden_size]
        e1 = sentence_embeddings
        e2 = aspect_embeddings.view(opt.batch_size,1,opt.hidden_size).expand(opt.batch_size,opt.max_seq_len,opt.hidden_size)
        
        # e1: [batch_size, max_seq_len, hidden_size]
        # e2: [batch_size, max_seq_len, hidden_size]
        
        wv = t.cat((e1,e2),dim=-1)
        # wv: [batch_size, max_seq_len, 2*hidden_size]
        
        out, (h, c) = self.lstm(wv, (self.h0, self.c0))
        # e.g.
        # out: [batch_size, max_seq_len, hidden_size]
        # h: [batch_size, 1, hidden_size]
        # c: [batch_size, 1, hidden_size]
        
        Wh_out = t.matmul(out, self.Wh)
        # [batch_size, max_seq_len, hidden_size]
        Wv_aspect = t.matmul(aspect_embeddings, self.Wv).view(opt.batch_size,1,opt.hidden_size).expand(opt.batch_size,opt.max_seq_len,opt.hidden_size)
        # [batch_size, max_seq_len, hidden_size]
        
        vh = t.cat((Wh_out, Wv_aspect), dim=2)
        # [batch_size, max_seq_len, 2*hidden_size]
        
        M = self.tanh(vh)
        # [batch_size, max_seq_len, 2*hidden_size]
        
        alpha = nn.functional.softmax(t.matmul(M, self.omega), dim=1).view(opt.batch_size,1,opt.max_seq_len)
        # [batch_size, 1, max_seq_len]
        
        r = t.matmul(alpha, out)
        # [batch_size, 1, hidden_size]
        
        _h_ = self.relu(t.matmul(t.squeeze(r), self.Wp) + t.matmul(t.squeeze(h), self.Wx))
        # [batch_size, hidden_size]
        
        y = nn.functional.softmax(self.lin(_h_), dim=1)
        # [batch_size, num_classes]
        
        return y

In [4]:
if __name__=='__main__':
    testDataset = AspClas(opt.test_data_root)
    testDataLoader = DataLoader(testDataset, batch_size=opt.batch_size, shuffle=True)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:14<00:00, 6975.44it/s]


Embedding : successfully input 100000 pretrained word embeddings while 0 failed


In [5]:
if __name__=='__main__':
    %pdb on
    model = ATAE_LSTM(testDataset.emb)
    sentence, terms, label = list(testDataLoader)[0]
    print(sentence)
    print(terms)
    print(label)
    y = model(sentence, terms)
    print(y)

Automatic pdb calling has been turned ON
tensor([[110,  81, 111,  66,  32,   5,  66,  63, 112,  79,  65,  86,   5, 113,
          89, 114,   3, 115, 116,  79,  66, 117,  20,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  2,   3,   4,   5,   6,   7,   8,   9,   3,  10,  11,  12,  13,   5,
           3,  14,  11,  15,  16,  16,  16,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [142,   3, 143,   5, 144,   7,  66,   9,   3,  63,   5,  32, 145,  89,
         146,  79, 147, 148,  20,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  3,  21,  29,  11,  22,  23,  11,   3,  24,  30,   5,   3,  25,  26,
          27,   3,  31,  11,  28,  20,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0]])
tensor([[86,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [10,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [32,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [24, 30,  0,  0,  0,  0,  0,  0,  0,  0]])
tenso